In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt



In [3]:
df_review = pd.read_csv("yelp_academic_dataset_review.csv")
df_review = pd.DataFrame(df_review)

df_business = pd.read_csv("yelp_academic_dataset_business.csv")
df_business = pd.DataFrame(df_business)

df_user = pd.read_csv("yelp_academic_dataset_user.csv")
df_user = pd.DataFrame(df_user)

C:\Users\asus\AppData\Local\Temp\ipykernel_8780\337022466.py:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_user = pd.read_csv("yelp_academic_dataset_user.csv")


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations

# Step 1: Filter businesses to include only Restaurants in Philadelphia
df_business_restaurants = df_business[
    df_business['categories'].str.contains("Restaurants", na=False) & 
    (df_business['city'] == 'Philadelphia')
]

# Step 2: Filter reviews for those businesses only
df_filtered_reviews = pd.merge(df_review, df_business_restaurants[['business_id']], on='business_id', how='inner')

# Step 3: Create a user interaction network
G = nx.Graph()

# Create edges between users who reviewed the same business using combinations
for business_id, group in df_filtered_reviews.groupby('business_id'):
    users = group['user_id'].tolist()
    # Use combinations to add edges
    for user1, user2 in combinations(users, 2):
        G.add_edge(user1, user2)

# Step 4: Add user information to the graph (optional, for future analysis)
# Create a mapping of user_id to user info
user_info = df_user.set_index('user_id')[['review_count', 'yelping_since']].to_dict(orient='index')

for user_id in G.nodes:
    if user_id in user_info:
        G.nodes[user_id].update(user_info[user_id])

# Step 5: Analyze and visualize the network
plt.figure(figsize=(10, 10))
nx.draw(G, with_labels=False, node_size=50, node_color='blue', edge_color='gray')
plt.title("User Interaction Network based on Reviews of Restaurants in Philadelphia")
plt.show()

# Step 6: Basic network statistics
print("Number of nodes (users):", G.number_of_nodes())
print("Number of edges (interactions):", G.number_of_edges())

# Optional: Find communities (clusters) or central users in the network
